## Capstone Project: Near real-time monitoring of a manufacturing production line
**Keywords:** <font color='green'>SQL, AWS (S3 bucket, Sagemaker-XGBoost, lambda function, API Gateway), Power BI (Streaming dashboard, API)</font>   

**Background:** The objective of this capstone project was to build and deploy a model to monitor in near real-time a customer-critical attribute of a product being manufactured at one of my company's facilities. For a variety of reasons, such limited resources and testing capabilities, this critical attribute can only be measured every 12 hours. Finished products are manufactured at 400-600 per minute rate. Therefore, a failure to meet this customer-critical attribute has the consequence of having to put on hold (often scrap) 12 hours of production. During these 12-hour periods several quality and production checks are performed at each stage of the product manufacturing process. The built model uses the data from these intermediary checks to predict the customer-critical attribute during the 12-hour intervals where this attribute is not measured directly. If a failure is predicted, a notification is sent to the appropriate personnel to take immediate action. 
#### Project structure:
- **Part I: ETL** The data used for training, validation and testing is hosted on two SQL servers. One SQL server host product quality data and the other SQL server host the machine state data. The first step is to extract the data from the SQL Servers, transform it and load it to an AWS S3 bucket. 
- **Part II:** 
 - **II.1 Build, Train and Deploy the model** With the data in AWS S3, I used Sagemaker to train and deploy an XGBoost model. Deploying the model creates an endpoint that can be accessed for predictions. 
 - **II.2 Lambda function & Gateway API** I created a lambda function & GateWay API to be able to access the model for predictions. The API allows me to the send the data for prediction as a POST request for low-latency response. This is a cost effective solution since I am only charged when I send the request to the API. 
- **Part III: Predict near real-time and stream to PowerBI dashboard** With the model deployed and the API in service, I scheduled a taks on one of our on-premises servers to send the latest intermediary check to the model and get a prediction of the customer-critical attribute. This prediction is then pass to the PowerBI dashboard (also as POST request). The PowerBI Streaming dashboard visualizes the predictions in real-time.   

### <font color='brown'>Part I:</font> Data transformation
* After quering the SQL databases with the data for training and testing I performed data transformation and feature engineering. 
* The output is saved to csv file and uploaded to a AWS S3 bucket for training the XGBoost model. 

In [ ]:
import pandas as pd
import xml.etree.ElementTree as ET
import matplotlib as plt
from sklearn import linear_model, ensemble
import numpy as np
# Setting pandas display options to show all rows
pd.options.display.max_rows = 200

In [ ]:
# Path to both the dat and nwh files
filenames = ['CAN - AXIAL LOAD', 
            'CAN - BACKEND FINISHED CAN', 
            'CAN - BEAD DEPTH',
            'CAN - PANEL RESISTANCE']
data_path = 'Axial_load/300x407/'

In [ ]:
# NWH Header files are XML files. I defined a function to parse the column names from the header file
def xmlParse(filename):
    path = ''.join([data_path, filename, '.NWH'])
    tree = ET.parse(path)
    root = tree.getroot()
    col_names = []
    for col in root.iter('ColumnParameter'):
        col_names.append('_'.join([col[0].text,col[1].text]))
    return col_names

In [ ]:
# Loading data from DAT file
data = []
for name in filenames:
    col_names = xmlParse(name) # Getting column names from NWH header file
    path = ''.join([data_path, name, '.DAT'])
    df = pd.read_csv(path, names=col_names, sep=' ')
    data.append(df)
 

In [ ]:
# Droping columns with no information. It would be good to start collecting Supplier & Raw ID data
for i in range(len(data)):
    col_to_drop = ['FACTORY_A',
                   'SET_NO_A',
                   'DEPT_A',
                   'GAGENAME_A',
                   'ENGRAVE_A', # they seem to only run 'Stock'
                   'SUPPLIER_A', # I want to collect this data to be able to use coil certificate data
                   'RAW_ID_A',  # I want to collect this data to be able to use coil certificate data
                   'NOTES_A',
                   'FILENAME_A',
                   'SPARE_1_A', 
                   'SPARE_2_A', 
                   'SPARE_3_A', 
                   'SPARE_4_A', 
                   'SAMPLESIZE_Z',
                   'CHKTYPE_A', # Only using production data
                   'SPECIFICATION_A', # It is only 1 spec
                   'CUSTOMER_A' # Everything is 'Stock'
                  ]
    data[i].drop(columns=col_to_drop, inplace=True)

In [ ]:
axial = data[0]
backend = data[1].drop(columns='CHECKNO_A')
bead = data[2].drop(columns=['ZLAST_I', 'QMDSTATION_N']) # 1 check per shift
panel = data[3]

In [ ]:
# One check per shift
axial.rename(columns = {'TIME_T': 'TIME_Axial', 'OPER_A': 'OPER_Axial'}, inplace=True)

In [ ]:
# 3 checks per shift
backend.rename(columns = {'TIME_T': 'TIME_BE', 'OPER_BE': 'OPER_BE'}, inplace=True)

In [ ]:
merge_on_cols = ['DATE_D', 'CREW_A', 'LINE_A', 'MACHINE_A', 'STATION_A', 'MS_NUMBER_A']
df = pd.merge(backend, axial, how='left', on=merge_on_cols)
df.head()

In [ ]:
bead.rename(columns={'TIME_T':'TIME_Bead','OPER_A':'OPER_Bead', 'FB_BEADER_TEMPERATURE_I': 'BEADER_TMP'}, inplace=True)

In [ ]:
merge_on_cols = ['DATE_D', 'CREW_A', 'LINE_A', 'MACHINE_A', 'STATION_A', 'MS_NUMBER_A']
df = pd.merge(df, bead, how='left', on=merge_on_cols)

In [ ]:
panel.rename(columns={'TIME_T':'TIME_panel', 'OPER_A':'OPER_bead'}, inplace=True)
panel.head(5)

In [ ]:
merge_on_cols = ['DATE_D', 'CREW_A', 'LINE_A', 'MACHINE_A', 'STATION_A', 'MS_NUMBER_A', 'FB_BODY_MAKER_A']
df = pd.merge(df, panel, how='left', on=merge_on_cols)

In [ ]:
cont_features = ['FBA__I', 
                 'FBH_AVG_I', 
                 'FBE_AVG_I', 
                 'FBD_AVG_I', 
                 'BEADER_TMP', 
                 'FBL_AVG_I', 
                 'FBL1AVG_I',
                'FBL2AVG_I',
                'FBL3AVG_I',
                 'FBL4AVG_I',
                 'FBL5AVG_I',
                 'FBL6AVG_I',
                 'FBL7AVG_I',
                 'FBL8AVG_I',
                 'FBL9AVG_I',
                 'FBL10AVG_I',
                 'FBL11AVG_I',
                 'FBL12AVG_I',
                 'FBL13AVG_I',
                 'FBL14AVG_I',
                 'FBL15AVG_I',
                 'FBL16AVG_I',
                 'FBL17AVG_I',
                 'FBL18AVG_I',
                 'FBL19AVG_I',
                 'FBAXIAL_I'
                 
                ]


In [ ]:
df.replace('*', np.NaN, inplace=True)

In [ ]:
df.shape

In [ ]:
# List of features with NA values
df.isnull().sum().sort_values(ascending=False).reset_index().rename(columns={0:'Feature'})

In [ ]:
# Droping columns with mostly NAs
df.drop(['OPER_bead', 
               'FBPANEL_I',
               'TIME_panel', 
               'CHECKNO_A_y',
               'CHECKNO_A', 
               'FBL19AVG_I',
               'FBL19_1', 
               'FBL19_2', 
               'FBL19_3', 
               'FBL19RNG_I'], axis=1, inplace=True)

In [ ]:
# Droping rows with NAs
df.dropna(inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
['DATE_D', 'TIME_BE', 'TIME_Axial', 'TIME_Bead']

In [ ]:
df.shape

In [ ]:
df.info(verbose=True)

In [ ]:
# Saving data to file
df.to_csv('./Axial_load/300x407/axial_for_training.csv')